# Consolidation Calculation

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sympy import symbols, Eq, solve, sympify
import pandas as pd
from sympy import symbols, Eq, solve


In [ ]:
def generate_matrix(dz, dt, ue):
    t_cols = [i*dt for i in range(dt)]
    z_rows = [j*dz for j in range(dz)]
    return pd.DataFrame(np.full((dz, dt), ue), columns=t_cols, index=z_rows)
bc = generate_matrix(5, 4, None)

,0,4,8,12
0,None,None,None,None
5,None,None,None,None
10,None,None,None,None
15,None,None,None,None
20,None,None,None,None


In [28]:
import numpy as np
import pandas as pd
from sympy import symbols, Eq, solve

def generate_matrix(dz, dt, fill=None, col_labels=None, row_labels=None):
    data = np.full((dz, dt), np.nan if fill is None else fill, dtype=float)
    cols = col_labels or [f"t{k}" for k in range(dt)]
    rows = row_labels or [f"z{j}" for j in range(dz)]
    return pd.DataFrame(data, index=rows, columns=cols)

class SoilLayer:
    def __init__(self, cv=None, del_z=None, beta=None):
        self.cv = cv        # m^2/s
        self.del_z = del_z  # m
        self.beta = beta    # dimensionless = cv*dt/dz^2
        self.del_t = None   # s
        self.boundary_conditions = None

    def solve_del_t(self):
        cv, del_t, del_z, beta = symbols('cv del_t del_z beta')
        eq = Eq(beta, cv * del_t / del_z**2)
        if self.cv is not None and self.del_z is not None and self.beta is not None:
            self.del_t = self.beta * (self.del_z ** 2) / self.cv
            return float(self.del_t)
        return solve(eq, del_t)[0]

    def solve_boundary_conditions(self, dz, dt, u0=64.0, top_drained_value=0.0):
        """
        Build the base matrix and apply BCs:
        - Column t0: initial uniform u_e = u0
        - Row z0, all t>0: drained, u = top_drained_value
        """
        M = generate_matrix(dz, dt, fill=None)
        # Initial condition at t0
        M.iloc[:, 0] = u0
        # Top drained boundary for t>=1
        M.iloc[0, 1:] = top_drained_value
        self.boundary_conditions = M
        return self.boundary_conditions
    def solve_terzaghi_fd(self, dz, dt, u0=64.0, top_drained_value=0.0,
                        stability_check=True, return_ghost=True):
        """
        Explicit FD for 1D Terzaghi consolidation on (dz x dt) grid.
        Bottom BC: zero gradient via ghost node u_{n+1}=u_{n-1}.
        If return_ghost=True, appends a visible 'z_ghost' row to the DataFrame.
        """
        if self.beta is None:
            raise ValueError("beta must be set (beta = cv*dt/dz^2).")
        if stability_check and self.beta > 0.5:
            raise ValueError(f"Unstable: beta={self.beta:.3f} > 0.5.")

        # Base matrix + BCs
        M = self.solve_boundary_conditions(dz, dt, u0=u0, top_drained_value=top_drained_value)

        # Prepare ghost-node storage (for visualization only)
        ghost = np.full(dt, np.nan)
        if dz == 1:
            ghost[0] = top_drained_value
        elif dz >= 2:
            # At t0, ghost equals the (n-1) node at t0
            ghost[0] = M.iloc[dz-2, 0]

        # Time-march
        for k in range(1, dt):
            prev = M.iloc[:, k-1].to_numpy()  # u(z, t_{k-1})
            new  = np.empty(dz, dtype=float)

            # Top Dirichlet
            new[0] = top_drained_value

            # Interior nodes
            if dz > 2:
                new[1:dz-1] = prev[1:dz-1] + self.beta*(prev[2:dz] - 2*prev[1:dz-1] + prev[0:dz-2])

            # Bottom node using ghost: u_{n+1}=u_{n-1}
            if dz == 1:
                new[0] = top_drained_value
            elif dz == 2:
                # u_2 uses u_3 = u_1 -> (prev[0] - 2*prev[1] + prev[0])
                new[1] = prev[1] + self.beta*(prev[0] - 2*prev[1] + prev[0])
            else:
                i = dz - 1
                new[i] = prev[i] + self.beta*(prev[i-1] - 2*prev[i] + prev[i-1])

            # Write time slice
            M.iloc[:, k] = new

            # Record ghost (at time t_k, the ghost equals current (n-1) node)
            if dz == 1:
                ghost[k] = top_drained_value
            else:
                ghost[k] = new[dz-2]

        # Make the ghost visible
        if return_ghost:
            M.loc["z_ghost"] = ghost

        return M


In [34]:
layer = SoilLayer(cv=1e-7, del_z=0.1, beta=0.5)
dz, dt = 5, 7
U = layer.solve_terzaghi_fd(dz, dt, u0=64.0, top_drained_value=0.0, return_ghost=True)
U

,t0,t1,t2,t3,t4,t5,t6
z0,64.0,0.0,0.0,0.0,0.0,0.0,0.0
z1,64.0,64.0,32.0,32.0,24.0,24.0,20.0
z2,64.0,64.0,64.0,48.0,48.0,40.0,40.0
z3,64.0,64.0,64.0,64.0,56.0,56.0,48.0
z4,64.0,64.0,64.0,64.0,64.0,56.0,56.0
z_ghost,64.0,64.0,64.0,64.0,56.0,56.0,48.0
